In [ ]:
!pip install -q  torch peft==0.5.0 bitsandbytes==0.40.2 trl==0.4.7 accelerate datasets

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


In [ ]:
import datasets

In [ ]:
# Dataset
data_name = "iprahara/text_to_cypher"
training_data = load_dataset(data_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:

training_data

Dataset({
    features: ['text'],
    num_rows: 149
})

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00


In [ ]:
# Model and tokenizer names
base_model_name = "codellama/CodeLlama-7b-hf"
refined_model = "codellama_refined"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# #Don't use this, extremely slow
# query = """Given my Neo4j database schema:[<Record nodes=[<Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>] relationships=[<Relationship element_id='-7' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='author_write_paper' properties={'name': 'author_write_paper'}>, <Relationship element_id='-8' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='paper_cite_paper' properties={'name': 'paper_cite_paper'}>, <Relationship element_id='-10' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>) type='paper_in_venue' properties={'name': 'paper_in_venue'}>, <Relationship element_id='-6' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>) type='author_in_affiliation' properties={'name': 'author_in_affiliation'}>, <Relationship element_id='-9' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>) type='paper_in_domain' properties={'name': 'paper_in_domain'}>]>] . Generate cypher query for the question: What are the top 5 main research areas in the 'ACL' conference? Only give cypher query, without any other words."""
# text_gen = pipeline(task="text-generation", model=base_model, tokenizer=llama_tokenizer, max_length=1024)
# output = text_gen(f"<s>[INST] {query} [/INST]")
# print(output[0]['generated_text'])

In [ ]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=10,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)

# Training
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

In [ ]:
# fine_tuning.model.save_pretrained(refined_model)


In [ ]:
# # tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# prompt = "Generate cypher query for the question: What are the top 5 main research areas in the 'ACL' conference? Only give cypher query, without any other words."
# # Generate predictions
# inputs = llama_tokenizer(prompt, return_tensors='pt')
# inputs = inputs.to("cuda")
# output = fine_tuning.model.generate(**inputs, max_new_tokens=100)
# response = llama_tokenizer.decode(output[0].tolist())

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
# response

"<s> Generate cypher query for the question: What are the top 5 main research areas in the 'ACL' conference? Only give cypher query, without any other words.\nOnly give cypher query, without any other words.\nMATCH (p:paper)-[:paper_in_venue]->(:conference {name: 'ACL'}),\n      (p)-[:paper_in_domain]->(d:domain)\nRETURN d.name, COUNT(p) AS numPapers\nORDER BY numPapers DESC\nLIMIT 5\nRETURN d.name, numPapers\nAS top5MainRes"

In [ ]:
import pandas as pd
df_test_all = pd.read_csv("PAML2_LLAMA2_result.csv")
df_test_all

,Unnamed: 0.1,Unnamed: 0,prompt,palm_generations,question,cypher,test,fine_tuned_palm_generations,fine_tuned_llama_generations,fine_tuned_llama_cleaned_generations,llama_generations,llama_cleaned_generations
0,0,0,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper) RETURN count(p),How many papers are in the database?,MATCH (p:paper) RETURN count(p) AS num_papers,1,MATCH (p:paper)\nRETURN COUNT(p),<s> Generate cypher query for the question: Ho...,MATCH (p:paper) RETURN count(p) AS num_papers,<s> Generate cypher query for the question: Ho...,MATCH (p:Paper) RETURN COUNT(p)\n\nPlease prov...
1,1,1,Given my Neo4j database schema:[<Record nodes=...,MATCH (a:author) RETURN count(a),Count the number of authors.,MATCH (a:Author) RETURN count(a) AS authorCount,1,MATCH (a:author)\nRETURN COUNT(a),<s> Generate cypher query for the question: Co...,MATCH (a:Author)\nRETURN count(a) AS authorCount,<s> Generate cypher query for the question: Co...,MATCH (a:Author)-[:WROTE]->(b:Book)\nWHERE LEN...
2,2,2,Given my Neo4j database schema:[<Record nodes=...,MATCH (c:conference) RETURN count(c),How many conferences are listed?,MATCH (c:conference) RETURN count(c) AS countC...,0,MATCH (c:conference)\nRETURN COUNT(c),<s> Generate cypher query for the question: Ho...,MATCH (c:conference) RETURN count(c) AS countC...,<s> Generate cypher query for the question: Ho...,[]
3,3,3,Given my Neo4j database schema:[<Record nodes=...,"MATCH (a:author {name: ""Daphne Koller""})-[:aut...",Find all papers by a 'Daphne Koller',MATCH (a:author {name:'Daphne Koller'})-[:auth...,0,MATCH (a:author {name: 'Daphne Koller'})-[:au...,<s> Generate cypher query for the question: Fi...,MATCH (a:author {name:'Daphne Koller'})-[:auth...,<s> Generate cypher query for the question: Fi...,[]
4,4,4,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[r:paper_cite_paper]->(p2:pape...,How many papers cite at least one other paper?,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,1,MATCH (p:paper)-[:paper_cite_paper]->(p2:paper...,<s> Generate cypher query for the question: Ho...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,<s> Generate cypher query for the question: Ho...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
214,214,214,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,Which papers on Maximum Likelihood have been p...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,<s> Generate cypher query for the question: Wh...,MATCH (p:paper)-[:paper_in_venue]->(:conferenc...,<s> Generate cypher query for the question: Wh...,[]
215,215,215,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain)...,Identify recent popular papers in Augmented Re...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_venue]->(c:conferen...,<s> Generate cypher query for the question: Id...,[],<s> Generate cypher query for the question: Id...,[]
216,216,216,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain)...,What are the trending research topics in Super...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (d:domain {name: 'Supervised Learning'}...,<s> Generate cypher query for the question: Wh...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,<s> Generate cypher query for the question: Wh...,[]
217,217,217,Given my Neo4j database schema:[<Record nodes=...,```\nMATCH (p:paper)-[:paper_in_domain]->(d:do...,List the latest papers in Bioinformatics with ...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,<s> Generate cypher query for the question: Li...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,<s> Generate cypher query for the question: Li...,[]


In [ ]:
df_test = df_test_all[df_test_all['test']==1][:20]

In [ ]:
df_test

,Unnamed: 0.1,Unnamed: 0,prompt,palm_generations,question,cypher,test,fine_tuned_palm_generations,fine_tuned_llama_generations,fine_tuned_llama_cleaned_generations,llama_generations,llama_cleaned_generations
0,0,0,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper) RETURN count(p),How many papers are in the database?,MATCH (p:paper) RETURN count(p) AS num_papers,1,MATCH (p:paper)\nRETURN COUNT(p),<s> Generate cypher query for the question: Ho...,MATCH (p:paper) RETURN count(p) AS num_papers,<s> Generate cypher query for the question: Ho...,MATCH (p:Paper) RETURN COUNT(p)\n\nPlease prov...
1,1,1,Given my Neo4j database schema:[<Record nodes=...,MATCH (a:author) RETURN count(a),Count the number of authors.,MATCH (a:Author) RETURN count(a) AS authorCount,1,MATCH (a:author)\nRETURN COUNT(a),<s> Generate cypher query for the question: Co...,MATCH (a:Author)\nRETURN count(a) AS authorCount,<s> Generate cypher query for the question: Co...,MATCH (a:Author)-[:WROTE]->(b:Book)\nWHERE LEN...
4,4,4,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[r:paper_cite_paper]->(p2:pape...,How many papers cite at least one other paper?,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,1,MATCH (p:paper)-[:paper_cite_paper]->(p2:paper...,<s> Generate cypher query for the question: Ho...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,<s> Generate cypher query for the question: Ho...,[]
5,5,5,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[r:paper_cite_paper]->(p2:pape...,How many papers are cited by at least one othe...,MATCH (other:paper)-[:paper_cite_paper]->(p:pa...,1,MATCH (p:paper)<-[:paper_cite_paper]-(citing:...,<s> Generate cypher query for the question: Ho...,MATCH (p:paper)<-[:paper_cite_paper]-(:paper)\...,<s> Generate cypher query for the question: Ho...,[]
6,6,6,Given my Neo4j database schema:[<Record nodes=...,```\nMATCH (p:paper)-[r:paper_cite_paper]->(c:...,How many papers are cited by at least ten othe...,MATCH (p:paper)<-[:paper_cite_paper]-(:paper)\...,1,MATCH (p:paper)<-[:paper_cite_paper]-(citing:p...,<s> Generate cypher query for the question: Ho...,MATCH (p:paper)<-[:paper_cite_paper]-(:paper)\...,<s> Generate cypher query for the question: Ho...,MATCH (p:Paper)\nWHERE p.citationCount > 10\nR...
8,8,8,Given my Neo4j database schema:[<Record nodes=...,MATCH (a:affiliation) RETURN a.name,Name all affiliations.,MATCH (a:affiliation)\nRETURN a.name,1,MATCH (a:affiliation)\nRETURN a.name,<s> Generate cypher query for the question: Na...,MATCH (a:affiliation)\nRETURN a.name,<s> Generate cypher query for the question: Na...,"MATCH (p:Publication {field_of_study: ""Compute..."
9,9,9,Given my Neo4j database schema:[<Record nodes=...,MATCH (d:domain) RETURN d.name AS domain,What are the different domains covered?,MATCH (d:domain)\nRETURN d.name,1,MATCH (d:domain)\nRETURN d.name,<s> Generate cypher query for the question: Wh...,MATCH (c:conference {name: 'ICML'})<-[:paper_i...,<s> Generate cypher query for the question: Wh...,[]
10,10,10,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain)...,What papers are Computer Science?,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,1,MATCH (p:paper)-[:paper_in_domain]->(d:domain...,<s> Generate cypher query for the question: Wh...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,<s> Generate cypher query for the question: Wh...,[]
12,12,12,Given my Neo4j database schema:[<Record nodes=...,MATCH (a:affiliation) RETURN count(a),What is the total number of affiliations?,MATCH (a:affiliation)\nRETURN COUNT(a),1,MATCH (a:affiliation)\nRETURN COUNT(a),<s> Generate cypher query for the question: Wh...,MATCH (a:affiliation)<-[:author_in_affiliation...,<s> Generate cypher query for the question: Wh...,[]
13,13,13,Given my Neo4j database schema:[<Record nodes=...,MATCH (a:author)-[:author_in_affiliation]->(c:...,List papers written by authors from Columbia U...,MATCH (a:author)-[:author_in_affiliation]->(af..

In [ ]:
[<Record nodes=[<Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>] relationships=[<Relationship element_id='-7' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='author_write_paper' properties={'name': 'author_write_paper'}>, <Relationship element_id='-8' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='paper_cite_paper' properties={'name': 'paper_cite_paper'}>, <Relationship element_id='-10' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>) type='paper_in_venue' properties={'name': 'paper_in_venue'}>, <Relationship element_id='-6' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>) type='author_in_affiliation' properties={'name': 'author_in_affiliation'}>, <Relationship element_id='-9' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>) type='paper_in_domain' properties={'name': 'paper_in_domain'}>]>]

In [ ]:
<Relationship element_id='-6' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>,

In [ ]:
schema= '''This schema describes a network of relationships within the academic domain. It includes five types of nodes: conference, paper, affiliation, author, and domain. Each node type represents a different entity: 'conference' for academic conferences, 'paper' for research papers, 'affiliation' for institutional affiliations of authors, 'author' for individuals who write papers, and 'domain' for areas of research.

There are five types of relationships defined in the schema:

'author_write_paper' links authors to the papers they have written.
'paper_cite_paper' represents citations from one paper to another.
'paper_in_venue' connects papers to the conferences where they are presented.
'author_in_affiliation' associates authors with their affiliated institutions.
'paper_in_domain' links papers to their respective research domains.'''

In [ ]:
from tqdm.notebook import tqdm
generations =[]
for i in tqdm(range(len(df_test)), "generating..."):
# i=1
# schema="<Relationship element_id='-6' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>"
  prompt = "Given the neo4j schema: "+schema+"\nGenerate cypher query for the question: "+schema+df_test_all.iloc[i]['question']+" Only give cypher query, without any other words. Put </s> at the end of answer."

  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = base_model.generate(**inputs, max_new_tokens=150,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  df_test['response']=response
  generations.append(response)
generations

generating...:   0%|          | 0/20 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

["<s> Given the neo4j schema: This schema describes a network of relationships within the academic domain. It includes five types of nodes: conference, paper, affiliation, author, and domain. Each node type represents a different entity: 'conference' for academic conferences, 'paper' for research papers, 'affiliation' for institutional affiliations of authors, 'author' for individuals who write papers, and 'domain' for areas of research.\n\nThere are five types of relationships defined in the schema:\n\n'author_write_paper' links authors to the papers they have written.\n'paper_cite_paper' represents citations from one paper to another.\n'paper_in_venue' connects papers to the conferences where they are presented.\n'author_in_affiliation' associates authors with their affiliated institutions.\n'paper_in_domain' links papers to their respective research domains.\nGenerate cypher query for the question: This schema describes a network of relationships within the academic domain. It inclu

In [ ]:
generations[2]

"<s> Given the neo4j schema: This schema describes a network of relationships within the academic domain. It includes five types of nodes: conference, paper, affiliation, author, and domain. Each node type represents a different entity: 'conference' for academic conferences, 'paper' for research papers, 'affiliation' for institutional affiliations of authors, 'author' for individuals who write papers, and 'domain' for areas of research.\n\nThere are five types of relationships defined in the schema:\n\n'author_write_paper' links authors to the papers they have written.\n'paper_cite_paper' represents citations from one paper to another.\n'paper_in_venue' connects papers to the conferences where they are presented.\n'author_in_affiliation' associates authors with their affiliated institutions.\n'paper_in_domain' links papers to their respective research domains.\nGenerate cypher query for the question: This schema describes a network of relationships within the academic domain. It includ

In [ ]:
response

"<s> Given the neo4j schema: <Relationship element_id='-6' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>\nGenerate cypher query for the question: <Relationship element_id='-6' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>Count the number of authors. Only give cypher query, without any other words. Put </s>  at the end of answer.\n\n### 1. What is the difference between a class and an object?\n\nA class is a blueprint for creating objects. An object is an instance of a class.\n\n### 2. What is the difference between a class and a struct?\n\nA class is a reference type, a struct is a value type.\n\n### 3. What is the difference between a class and a module?\n\nA class is a blueprint for creating objects. A module is a collection of related functions and variables.\n\n### 4. What is the difference between a class and a function?\n\nA cla

In [ ]:
response

'<s> Generate cypher query for the question: Cypher schema of the dataset to refer labels=frozenset({\'paper\'}) properties={\'name\': \'paper\', \'indexes\': [], \'constraints\': []}>Count the number of authors. Only give cypher query, without any other words.\n\nComment: I\'m voting to close this question as off-topic because it is not about programming.\n\nAnswer: \\begin{code}\nMATCH (a:Author)-[:AUTHOR_OF]->(p:Paper)\nWITH a, count(p) as c\nWHERE c > 1\nRETURN a.name, c\n\\end{code}\n\nComment: Thanks for your answer. I have tried your code, but it gives me the error: "Type mismatch: expected List<Map> but was Set<Map>"\n\nComment: I\'m not sure what you mean by "without any other words".\n\nComment: I mean that I only'

In [ ]:
df_test_all.iloc[10]['fine_tuned_llama_generations']

"<s> Generate cypher query for the question: What papers are Computer Science? Only give cypher query, without any other words.\n</a>\nMATCH (p:paper)-[:paper_in_domain]->(d:domain {name: 'Computer Science'})\nRETURN p.name</s></s>"

In [ ]:
pattern = r'MATCH.*?</s>'
df_test_all['fine_tuned_llama_cleaned_generations']= df_test_all['fine_tuned_llama_generations'].apply(
    lambda x:re.findall(pattern, x, re.DOTALL) )

In [ ]:
import re

pattern = r'MATCH.*?</s>'
text = """<s> Generate cypher query for the question: What papers are Computer Science? Only give cypher query, without any other words.
</a>
MATCH (p:paper)-[:paper_in_domain]->(d:domain {name: 'Computer Science'})
RETURN p.name</s></s>"""

matches = re.findall(pattern, text, re.DOTALL)[0].replace('</s>', '')

In [ ]:
matches

"MATCH (p:paper)-[:paper_in_domain]->(d:domain {name: 'Computer Science'})\nRETURN p.name</s>"

In [ ]:
df_test_all['fine_tuned_llama_cleaned_generations']

0      [MATCH (p:paper) RETURN count(p) AS num_papers...
1      [MATCH (a:Author)\nRETURN count(a) AS authorCo...
2      [MATCH (c:conference) RETURN count(c) AS count...
3      [MATCH (a:author {name:'Daphne Koller'})-[:aut...
4      [MATCH (p:paper)-[:paper_cite_paper]->(:paper)...
                             ...                        
214    [MATCH (p:paper)-[:paper_in_venue]->(:conferen...
215                                                   []
216    [MATCH (p:paper)-[:paper_in_domain]->(d:domain...
217    [MATCH (p:paper)-[:paper_in_domain]->(d:domain...
218                                                   []
Name: fine_tuned_llama_cleaned_generations, Length: 219, dtype: object

In [ ]:
df_test_all['fine_tuned_llama_cleaned_generations'] = df_test_all['fine_tuned_llama_cleaned_generations'].apply(
    lambda x: x[0].replace("</s>", '') if len(x)>0 else x)

In [ ]:
df_test_all['fine_tuned_llama_cleaned_generations']

0          MATCH (p:paper) RETURN count(p) AS num_papers
1       MATCH (a:Author)\nRETURN count(a) AS authorCount
2      MATCH (c:conference) RETURN count(c) AS countC...
3      MATCH (a:author {name:'Daphne Koller'})-[:auth...
4      MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...
                             ...                        
214    MATCH (p:paper)-[:paper_in_venue]->(:conferenc...
215                                                   []
216    MATCH (p:paper)-[:paper_in_domain]->(d:domain ...
217    MATCH (p:paper)-[:paper_in_domain]->(d:domain ...
218                                                   []
Name: fine_tuned_llama_cleaned_generations, Length: 219, dtype: object

In [ ]:
df_test_all.to_csv('PALM2_LLAMA2_results.csv')

In [ ]:
schema="""[<Record nodes=[<Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>] relationships=[<Relationship element_id='-7' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='author_write_paper' properties={'name': 'author_write_paper'}>, <Relationship element_id='-8' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='paper_cite_paper' properties={'name': 'paper_cite_paper'}>, <Relationship element_id='-10' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>) type='paper_in_venue' properties={'name': 'paper_in_venue'}>, <Relationship element_id='-6' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>) type='author_in_affiliation' properties={'name': 'author_in_affiliation'}>, <Relationship element_id='-9' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>) type='paper_in_domain' properties={'name': 'paper_in_domain'}>]>]"""

In [ ]:
from tqdm.notebook import tqdm
generations_original =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = "Given the neo4j schema: "+schema+"\nGenerate cypher query for the question: "+df_test_all.iloc[i]['question']+" Only give cypher query, without any other words. Put </s> at the end of answer."
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = base_model.generate(**inputs, max_new_tokens=150,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  # print(response)
  # break
  generations_original.append(response)
generations_original

generating...:   0%|          | 0/219 [00:00<?, ?it/s]

In [ ]:
df_test_all['llama_generations'] = generations_original

In [ ]:
pattern = r'MATCH.*?</s>'
df_test_all['llama_cleaned_generations']= df_test_all['llama_generations'].apply(
    lambda x:re.findall(pattern, x, re.DOTALL) )

df_test_all['llama_cleaned_generations'] = df_test_all['llama_cleaned_generations'].apply(
    lambda x: x[0].replace("</s>", '') if len(x)>0 else x)


In [ ]:
df_test_all.to_csv("PAML2_LLAMA2_results1.csv")
df_test_all.to_csv("/content/drive/MyDrive/cypher/PAML2_LLAMA2_results.csv")

In [ ]:
output = base_model.generate(**inputs, max_new_tokens=100,temperature=0.2)

In [ ]:
response = llama_tokenizer.decode(output[0].tolist())
response

'<s> Generate cypher query for the question: What is the total number of affiliations? Only give cypher query, without any other words. nobody has more than 5 affiliations.\n\nHere is the cypher query:\n```\nMATCH (p:Person)-[:AFFILIATION]->()\nRETURN p.affiliations.length\n```\nExplanation:\n\n* `MATCH` is used to find all the people in the graph.\n* `-[:AFFILIATION]->()` is a path query that finds all the affiliations of each person.\n* `RET'

In [ ]:
for i in range(len(df_test)):
  df_test.iloc[0]['answer']=  generations[i]

<ipython-input-27-ef16161d1e5f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.iloc[0]['answer']=  generations[i]


In [ ]:
df_test['answer']=generations

In [ ]:
df_test

,Unnamed: 0.1,Unnamed: 0,prompt,palm_generations,question,cypher,test,fine_tuned_palm_generations,fine_tuned_llama_generations,fine_tuned_llama_cleaned_generations,llama_generations,llama_cleaned_generations,response,answer
0,0,0,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper) RETURN count(p),How many papers are in the database?,MATCH (p:paper) RETURN count(p) AS num_papers,1,MATCH (p:paper)\nRETURN COUNT(p),<s> Generate cypher query for the question: Ho...,MATCH (p:paper) RETURN count(p) AS num_papers,<s> Generate cypher query for the question: Ho...,MATCH (p:Paper) RETURN COUNT(p)\n\nPlease prov...,<s> Given the neo4j schema: This schema descri...,<s> Given the neo4j schema: This schema descri...
1,1,1,Given my Neo4j database schema:[<Record nodes=...,MATCH (a:author) RETURN count(a),Count the number of authors.,MATCH (a:Author) RETURN count(a) AS authorCount,1,MATCH (a:author)\nRETURN COUNT(a),<s> Generate cypher query for the question: Co...,MATCH (a:Author)\nRETURN count(a) AS authorCount,<s> Generate cypher query for the question: Co...,MATCH (a:Author)-[:WROTE]->(b:Book)\nWHERE LEN...,<s> Given the neo4j schema: This schema descri...,<s> Given the neo4j schema: This schema descri...
4,4,4,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[r:paper_cite_paper]->(p2:pape...,How many papers cite at least one other paper?,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,1,MATCH (p:paper)-[:paper_cite_paper]->(p2:paper...,<s> Generate cypher query for the question: Ho...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,<s> Generate cypher query for the question: Ho...,[],<s> Given the neo4j schema: This schema descri...,<s> Given the neo4j schema: This schema descri...
5,5,5,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[r:paper_cite_paper]->(p2:pape...,How many papers are cited by at least one othe...,MATCH (other:paper)-[:paper_cite_paper]->(p:pa...,1,MATCH (p:paper)<-[:paper_cite_paper]-(citing:...,<s> Generate cypher query for the question: Ho...,MATCH (p:paper)<-[:paper_cite_paper]-(:paper)\...,<s> Generate cypher query for the question: Ho...,[],<s> Given the neo4j schema: This schema descri...,<s> Given the neo4j schema: This schema descri...
6,6,6,Given my Neo4j database schema:[<Record nodes=...,```\nMATCH (p:paper)-[r:paper_cite_paper]->(c:...,How many papers are cited by at least ten othe...,MATCH (p:paper)<-[:paper_cite_paper]-(:paper)\...,1,MATCH (p:paper)<-[:paper_cite_paper]-(citing:p...,<s> Generate cypher query for the question: Ho...,MATCH (p:paper)<-[:paper_cite_paper]-(:paper)\...,<s> Generate cypher query for the question: Ho...,MATCH (p:Paper)\nWHERE p.citationCount > 10\nR...,<s> Given the neo4j schema: This schema descri...,<s> Given the neo4j schema: This schema descri...
8,8,8,Given my Neo4j database schema:[<Record nodes=...,MATCH (a:affiliation) RETURN a.name,Name all affiliations.,MATCH (a:affiliation)\nRETURN a.name,1,MATCH (a:affiliation)\nRETURN a.name,<s> Generate cypher query for the question: Na...,MATCH (a:affiliation)\nRETURN a.name,<s> Generate cypher query for the question: Na...,"MATCH (p:Publication {field_of_study: ""Compute...",<s> Given the neo4j schema: This schema descri...,<s> Given the neo4j schema: This schema descri...
9,9,9,Given my Neo4j database schema:[<Record nodes=...,MATCH (d:domain) RETURN d.name AS domain,What are the different domains covered?,MATCH (d:domain)\nRETURN d.name,1,MATCH (d:domain)\nRETURN d.name,<s> Generate cypher query for the question: Wh...,MATCH (c:conference {name: 'ICML'})<-[:paper_i...,<s> Generate cypher query for the question: Wh...,[],<s> Given the neo4j schema: This schema descri...,<s> Given the neo4j schema: This schema descri...
10,10,10,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain)...,What papers are Computer Science?,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,1,MATCH (p:paper)-[:paper_in_domain]->(d:domain...,<s> Generate cyp

In [ ]:
!pip install nltk rouge -q
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

NotImplementedError: ignored

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
pip install nltk

In [ ]:
!pip install rouge -q
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

In [ ]:
import numpy as np
def calculate_bleu_rouge(reference, candidate):
    """
    Calculate BLEU and ROUGE scores for a pair of strings.

    :param reference: The reference string (ground truth).
    :param candidate: The candidate string (generated text).
    :return: A dictionary containing BLEU and ROUGE scores.
    """
    # Preprocessing the strings: tokenizing into words
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Calculating BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)

    # Calculating ROUGE score
    rouge = Rouge()
    rouge_score = rouge.get_scores(candidate, reference)[0]

    return np.array([bleu_score,rouge_score['rouge-1']['r'], rouge_score['rouge-2']['r'], rouge_score['rouge-l']['r']])

# Example usage



In [ ]:
scores_all = np.zeros((1,4))
for i in tqdm(range(len(df_test)), "evaluating..."):
  row = df_test.iloc[i]
  palm2 = row['answer']
  cyphers  = [palm2]
  label = row['cypher']
  for j, cy in enumerate(cyphers):
    scores_all[j]+= calculate_bleu_rouge(cy, label)

evaluating...:   0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:

scores_all = scores_all/len(df_test)

In [ ]:
df_nlp_score = pd.DataFrame(scores_all.T, columns = ['CodeLLama'])
df_nlp_score.index = ["BLEU", "ROUGE@1",  "ROUGE@2", "ROUGE@L"]
df_nlp_score

,CodeLLama
BLEU,1.185357e-238
ROUGE@1,4.725401e-03
ROUGE@2,0.000000e+00
ROUGE@L,4.725401e-03


In [ ]:
df_test.to_csv('codellama.csv')